In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score

2023-05-18 14:15:15.987817: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("training_set.tsv", sep='\t', encoding='ISO-8859-1');
df.dropna(axis=1,inplace=True)
df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
df.head()
temp = pd.read_csv("Processed_data.csv")
temp.drop("Unnamed: 0",inplace=True,axis=1)

In [3]:
df['domain1_score']=temp['final_score']
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,8,A long time ago when I was in third grade I ha...,6
1,2,8,Softball has to be one of the single most grea...,8
2,3,8,"Some people like making people laugh, I love ...",7
3,4,8,"""LAUGHTER"" @CAPS1 I hang out with my friends,...",5
4,5,8,Well ima tell a story about the time i got @CA...,4


In [4]:
df['essay'][0]

"A long time ago when I was in third grade I had a friend @PERSON2 who's mom was in a bad mood. She never laughed and she never smiled. Every time I saw her I would smile at her and all she would do was frown and keep walking. At first I didn't know she was a grouch i just thought she didn't like me or something.When @PERSON2 told me his mom was a grouch I started to laugh and laugh. He asked me what was so funny i told him that i thought his mom didn't like me or something because every time I see his mom I would smile at her and all she will do is frown and walk away. That made my friend laugh we were cracking up so hard that we got in trouble in class.   The next day @PERSON2 and I were eating lunch at school when he says to me &lt;hey your pretty good at making people laugh&gt;. I said &lt;no I am not my jokesare horrible&gt;. He said &lt;@CAPS1 lets put them to the test go up to some one new to this school&gt;. I said @CAPS1 so we went around the whole school looking for a new stu

In [5]:
temp.head()

,essay_id,essay_set,essay,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,1,8,A long time ago when I was in third grade I ha...,6,A long time ago I third grade I friend s mom b...,2364,674,36,3.507418,59,134,158,49,26
1,2,8,Softball has to be one of the single most grea...,8,Softball one single greatest sports alive pla...,3067,779,28,3.937099,18,142,163,52,71
2,3,8,"Some people like making people laugh, I love i...",7,Some people like making people laugh I love ...,3426,861,39,3.979094,31,161,174,57,62
3,4,8,"""LAUGHTER"" I hang out with my friends, the one...",5,LAUGHTER I hang friends one thing best laug...,2547,697,30,3.654232,19,124,155,31,63
4,5,8,Well ima tell a story about the time i got dow...,4,Well ima tell story time got town When headin...,2093,618,24,3.386731,50,120,174,41,76


In [6]:
#Make Dataset
y = df['domain1_score']
df.drop('domain1_score',inplace=True,axis=1)
X=df

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
X_train.shape

(506, 3)

**PREPROCESSING**

In [9]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()

In [10]:
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english')) 
def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)


In [11]:
len(train_sents)

17388

In [12]:
train_sents[0]

['In', 'ninth', 'grade', 'I', 'required', 'give', 'speech', 'book']

**Preparing WORD2VEC and LSTM Model**

In [13]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

In [14]:
#Training Word2Vec model
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

model = Word2Vec(train_sents, 
                 workers=num_workers, 
                 vector_size=num_features, 
                 min_count = min_word_count, 
                 window = context, 
                 sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

/var/folders/kn/gnm3d9hn21d8t3q08syhwsdw0000gp/T/ipykernel_6915/1462232201.py:15: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [15]:
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.wv.index_to_key)
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec,model.wv.get_vector(i))        
    vec = np.divide(vec,noOfWords)
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs


clean_train=[]
for i in train_e:
    clean_train.append(sent2word(i))
training_vectors = getVecs(clean_train, model, num_features)

clean_test=[] 

for i in test_e:
    clean_test.append(sent2word(i))
testing_vectors = getVecs(clean_test, model, num_features)

In [16]:
training_vectors.shape

(506, 300)

In [17]:
training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

2023-05-18 14:15:52.822292: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 300)            721200    
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________


In [18]:
training_vectors.shape

(506, 1, 300)

**TRAINING AND PREDICTION**

In [19]:
lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=150)

Epoch 1/150
8/8 [==============================] - 4s 14ms/step - loss: 35.9734 - mae: 5.8980
Epoch 2/150
8/8 [==============================] - 0s 14ms/step - loss: 16.8137 - mae: 3.8661
Epoch 3/150
8/8 [==============================] - 0s 14ms/step - loss: 4.0861 - mae: 1.7159
Epoch 4/150
8/8 [==============================] - 0s 14ms/step - loss: 2.1039 - mae: 1.1662
Epoch 5/150
8/8 [==============================] - 0s 14ms/step - loss: 1.8953 - mae: 1.0873
Epoch 6/150
8/8 [==============================] - 0s 14ms/step - loss: 1.6514 - mae: 1.0257
Epoch 7/150
8/8 [==============================] - 0s 14ms/step - loss: 2.0259 - mae: 1.1074
Epoch 8/150
8/8 [==============================] - 0s 15ms/step - loss: 1.7436 - mae: 1.0545
Epoch 9/150
8/8 [==============================] - 0s 14ms/step - loss: 1.9063 - mae: 1.0905
Epoch 10/150
8/8 [==============================] - 0s 13ms/step - loss: 1.7864 - mae: 1.0567
Epoch 11/150
8/8 [==============================] - 0s 12ms/step - 

In [20]:
lstm_model.save('final_lstm.h5')
y_pred = lstm_model.predict(testing_vectors)
y_pred = np.around(y_pred)
y_pred

7/7 [==============================] - 0s 4ms/step


array([[6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],